[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_model_comparison/EMQA_model_comparison.ipynb)

# EMQA_model_comparison

Rolling 1-step-ahead model comparison (Naive, Random Forest, Gradient Boosting, LSTM) on Romanian electricity price data with bootstrap confidence intervals. LSTM is trained using PyTorch.

**Output:** `ml_model_comparison.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")

In [ ]:
url = 'https://raw.githubusercontent.com/QuantLet/EMQA/main/EMQA_model_comparison/ro_de_prices_full.csv'
ro = pd.read_csv(url, parse_dates=['date'], index_col='date')
print(f'Loaded {len(ro)} observations')
print(ro.columns.tolist())
ro.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- Build 32-feature set predicting price CHANGES (same as EMQA_feature_importance) ---
data = ro.copy()
data['target'] = data['ro_price'].diff()  # predict daily price change

# RO lagged levels and changes
for lag in [1, 2, 3, 5, 7, 14, 30]:
    data[f'ro_lag_{lag}'] = data['ro_price'].shift(lag)
    data[f'ro_diff_lag_{lag}'] = data['ro_price'].diff().shift(lag)

# Rolling stats
for w in [7, 14, 30]:
    data[f'ro_ma_{w}'] = data['ro_price'].shift(1).rolling(w).mean()
    data[f'ro_std_{w}'] = data['ro_price'].shift(1).rolling(w).std()

# DE cross-market features
for lag in [1, 7]:
    data[f'de_lag_{lag}'] = data['de_price'].shift(lag)
data['spread_lag1'] = data['ro_price'].shift(1) - data['de_price'].shift(1)

# Temperature
data['ro_temp_lag1'] = data['ro_temp_mean'].shift(1)
data['hdd'] = (18 - data['ro_temp_mean'].shift(1)).clip(lower=0)
data['cdd'] = (data['ro_temp_mean'].shift(1) - 18).clip(lower=0)

# Consumption
data['consumption_lag1'] = data['ro_consumption'].shift(1)
data['consumption_lag7'] = data['ro_consumption'].shift(7)
data['residual_load_lag1'] = data['ro_residual_load'].shift(1)

# Temporal
data['dow'] = data.index.dayofweek
data['month'] = data.index.month
data['weekend'] = (data.index.dayofweek >= 5).astype(int)

data = data.dropna()
exclude = ['target', 'ro_price', 'de_price', 'gas_price',
           'de_temp_mean', 'de_temp_max', 'de_temp_min',
           'ro_temp_mean', 'ro_temp_max', 'ro_temp_min',
           'ro_nuclear', 'ro_hydro', 'ro_coal', 'ro_gas',
           'ro_wind', 'ro_solar', 'ro_consumption', 'ro_residual_load']
feature_cols = [c for c in data.columns if c not in exclude]

print(f"Dataset: {len(data)} rows, {len(feature_cols)} features")
print(f"Features: {feature_cols}")

In [ ]:
# Rolling expanding-window forecast: predict price CHANGES, convert to levels
init_train = int(len(data) * 0.6)
retrain_every = 30

rf_model = None
gb_model = None

# Storage (all in price LEVELS after conversion)
naive_preds, rf_preds, gb_preds = [], [], []
rf_ci_lo, rf_ci_hi = [], []
gb_ci_lo, gb_ci_hi = [], []
actuals, dates_out = [], []

for i in range(init_train, len(data)):
    step = i - init_train

    # Retrain RF and GB every 30 steps
    if step % retrain_every == 0:
        X_tr = data[feature_cols].iloc[:i].values
        y_tr = data['target'].iloc[:i].values  # price changes
        rf_model = RandomForestRegressor(
            n_estimators=200, max_depth=10, random_state=42, n_jobs=-1).fit(X_tr, y_tr)
        gb_model = GradientBoostingRegressor(
            n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42).fit(X_tr, y_tr)

    X_step = data[feature_cols].iloc[i:i+1].values
    prev_price = data['ro_price'].iloc[i - 1]
    actual_price = data['ro_price'].iloc[i]

    # Naive: tomorrow = today (zero change)
    naive_preds.append(prev_price)

    # RF prediction: change → level
    rf_chg = rf_model.predict(X_step)[0]
    rf_tree_chg = np.array([t.predict(X_step)[0] for t in rf_model.estimators_])
    rf_preds.append(prev_price + rf_chg)
    rf_ci_lo.append(prev_price + np.percentile(rf_tree_chg, 2.5))
    rf_ci_hi.append(prev_price + np.percentile(rf_tree_chg, 97.5))

    # GB prediction: change → level
    gb_chg = gb_model.predict(X_step)[0]
    gb_staged = np.array([p[0] for p in gb_model.staged_predict(X_step)])
    half = max(1, gb_model.n_estimators // 2)
    gb_recent = gb_staged[half:]
    gb_preds.append(prev_price + gb_chg)
    gb_ci_lo.append(prev_price + np.percentile(gb_recent, 2.5))
    gb_ci_hi.append(prev_price + np.percentile(gb_recent, 97.5))

    actuals.append(actual_price)
    dates_out.append(data.index[i])

# Convert
actuals = np.array(actuals)
dates_out = pd.DatetimeIndex(dates_out)
naive_preds = np.array(naive_preds)
rf_preds = np.array(rf_preds)
gb_preds = np.array(gb_preds)
rf_ci_lo = np.array(rf_ci_lo)
rf_ci_hi = np.array(rf_ci_hi)
gb_ci_lo = np.array(gb_ci_lo)
gb_ci_hi = np.array(gb_ci_hi)

# All results in levels; use R2_OOS = 1 - MSE_model/MSE_naive
all_models = {
    'Naive (lag-1)': naive_preds,
    'Random Forest': rf_preds,
    'Gradient Boosting': gb_preds,
}

results = {}
mse_naive = mean_squared_error(actuals, naive_preds)
for name, pred in all_models.items():
    mae = mean_absolute_error(actuals, pred)
    r2_oos = 1 - mean_squared_error(actuals, pred) / mse_naive
    results[name] = {'MAE': mae, 'R2_OOS': r2_oos}
    print(f"{name:22s}  MAE={mae:.2f}  R2_OOS={r2_oos*100:.1f}%")

best_name = min(
    [k for k in results if k != 'Naive (lag-1)'],
    key=lambda k: results[k]['MAE'])
print(f"\nBest tree model by MAE: {best_name}")

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

# --- Multivariate LSTM using PyTorch ---
LOOKBACK = 14
HIDDEN = 64
NUM_LAYERS = 2
EPOCHS = 100
LR = 0.003

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=HIDDEN, num_layers=NUM_LAYERS):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=0.1 if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :]).squeeze(-1)

def make_sequences_mv(features, targets, lookback):
    """Sequence for target[i] uses features[i-L+1:i+1] (includes current step)."""
    X, y = [], []
    for i in range(lookback - 1, len(features)):
        X.append(features[i - lookback + 1:i + 1])
        y.append(targets[i])
    return np.array(X), np.array(y)

def train_lstm_mv(features, targets, lookback=LOOKBACK, epochs=EPOCHS):
    """Train multivariate LSTM on feature sequences."""
    scaler_X = StandardScaler()
    feat_scaled = scaler_X.fit_transform(features)
    X_seq, y_seq = make_sequences_mv(feat_scaled, targets, lookback)
    X_t = torch.FloatTensor(X_seq)
    y_t = torch.FloatTensor(y_seq)
    model = LSTMModel(input_size=features.shape[1], hidden_size=HIDDEN, num_layers=NUM_LAYERS)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    loss_fn = nn.MSELoss()
    model.train()
    for _ in range(epochs):
        optimizer.zero_grad()
        loss = loss_fn(model(X_t), y_t)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    model.eval()
    return model, scaler_X

def predict_lstm_mv(model, scaler_X, recent_features, lookback=LOOKBACK):
    """Predict next change from recent feature matrix (last lookback rows)."""
    scaled = scaler_X.transform(recent_features[-lookback:])
    X = torch.FloatTensor(scaled).unsqueeze(0)  # (1, lookback, n_features)
    with torch.no_grad():
        return model(X).item()

# Rolling LSTM predictions (retrain every 30 steps, same as RF/GB)
torch.manual_seed(42)
np.random.seed(42)

lstm_model, lstm_scaler = None, None
lstm_preds = []
all_features = data[feature_cols].values
all_targets = data['target'].values  # price changes

for i in range(init_train, len(data)):
    step = i - init_train
    if step % retrain_every == 0:
        lstm_model, lstm_scaler = train_lstm_mv(all_features[:i], all_targets[:i])

    # Predict change, convert to level
    lstm_chg = predict_lstm_mv(lstm_model, lstm_scaler, all_features[:i+1])
    prev_price = data['ro_price'].iloc[i - 1]
    lstm_preds.append(prev_price + lstm_chg)

lstm_preds = np.array(lstm_preds)

# Ensemble: simple average of RF + GB + LSTM (level predictions)
ens_preds = (rf_preds + gb_preds + lstm_preds) / 3

all_models['LSTM'] = lstm_preds
all_models['Ensemble'] = ens_preds

for name in ['LSTM', 'Ensemble']:
    pred = all_models[name]
    mae = mean_absolute_error(actuals, pred)
    r2_oos = 1 - mean_squared_error(actuals, pred) / mse_naive
    results[name] = {'MAE': mae, 'R2_OOS': r2_oos}

# Direction accuracy for each model
print("\n=== Final Model Comparison ===")
for name in ['Naive (lag-1)', 'Random Forest', 'Gradient Boosting', 'LSTM', 'Ensemble']:
    pred = all_models[name]
    mae = results[name]['MAE']
    r2_oos = results[name]['R2_OOS']
    dir_correct = np.mean(np.sign(pred - naive_preds) == np.sign(actuals - naive_preds)) * 100
    results[name]['Direction'] = dir_correct
    print(f"  {name:22s}  MAE={mae:.2f}  R2_OOS={r2_oos*100:.1f}%  Dir={dir_correct:.0f}%")

In [ ]:
# Plot 1: Actual vs best model forecast with CI
if best_name == 'Random Forest':
    best_pred, best_lo, best_hi = rf_preds, rf_ci_lo, rf_ci_hi
elif best_name == 'Gradient Boosting':
    best_pred, best_lo, best_hi = gb_preds, gb_ci_lo, gb_ci_hi
else:
    best_pred, best_lo, best_hi = naive_preds, rf_ci_lo, rf_ci_hi

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dates_out, actuals, color=COLORS['blue'], lw=1.5, label='Actual')
ax.plot(dates_out, best_pred, color=COLORS['red'], lw=1.5, ls='--',
        label=f'{best_name} Forecast')
ax.fill_between(dates_out, best_lo, best_hi,
                color=COLORS['red'], alpha=0.12, label='95% CI')

ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.set_title(f'Romanian Electricity: Rolling 1-Step-Ahead ({best_name})\n'
             f'MAE={results[best_name]["MAE"]:.2f}, R²_OOS={results[best_name]["R2_OOS"]*100:.1f}%')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: Bar chart MAE and R2_OOS comparison (all models incl. LSTM + Ensemble)
plot_order = ['Naive (lag-1)', 'Random Forest', 'Gradient Boosting', 'LSTM', 'Ensemble']
res_df = pd.DataFrame(results).T.loc[plot_order]
bar_colors = [COLORS['gray'], COLORS['green'], COLORS['orange'], COLORS['purple'], COLORS['red']]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# (A) MAE
ax = axes[0]
bars = ax.bar(res_df.index, res_df['MAE'], color=bar_colors, alpha=0.8,
              edgecolor='white', lw=1.5)
for bar, val in zip(bars, res_df['MAE']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            f'{val:.1f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
ax.set_title('(A) Mean Absolute Error', fontsize=13, fontweight='bold')
ax.set_ylabel('MAE (EUR/MWh)')
ax.tick_params(axis='x', rotation=20)

# (B) R2_OOS (Coefficient of Determination vs naive)
ax2 = axes[1]
r2_vals = res_df['R2_OOS'] if 'R2_OOS' in res_df.columns else res_df['R2']
bars2 = ax2.bar(res_df.index, r2_vals, color=bar_colors, alpha=0.8,
                edgecolor='white', lw=1.5)
for bar, val in zip(bars2, r2_vals):
    ax2.text(bar.get_x() + bar.get_width()/2, max(val, 0) + 0.005,
             f'{val:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
ax2.set_title('(B) Coefficient of Determination', fontsize=13, fontweight='bold')
ax2.set_ylabel('R$^2_{OOS}$')
ax2.tick_params(axis='x', rotation=20)

fig.suptitle('Rolling Model Comparison: Romanian Electricity Price Forecasting',
             fontsize=15, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'ml_model_comparison.pdf')
plt.show()